In [1]:
%pip install pandas

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd

# 'shift-jis' エンコーディングを指定してファイルを読み込む
df = pd.read_csv(r".\\datafiles\\cinema.csv", encoding="shift-jis")
df.head(3)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x80 in position 0: invalid start byte

In [ ]:
#欠損値の確認
df.isnull().any(axis=0)

In [ ]:
#欠損値の穴埋め
df2 = df.fillna(df.mean())
df2.isnull().any(axis=0)

In [ ]:
#SNS2列とsales列の散布図を作成
%matplotlib inline
df2.plot(kind="scatter", x = "SNS2", y = "sales")

In [ ]:
#特徴量との組み合わせを変えて散布図を作成
df2.plot(kind ="scatter", x ="SNS1", y = "sales")
df2.plot(kind ="scatter", x ="SNS2", y = "sales")
df2.plot(kind ="scatter", x ="actor", y = "sales")
df2.plot(kind ="scatter", x ="original", y = "sales")

In [ ]:
#コード6-5と同じことをfor分との組み合わせで行う
for name in df.columns:
    if name == "cinema_id" or name == "sales":
        continue
    df.plot(kind="scatter",x = name ,y = "sales")

In [ ]:
#外れ値を削除する
no = df2[(df2['SNS2']>1000) & (df2["sales"]<8500)].index
df3 = df2.drop(no,axis=0)

In [ ]:
#データフレームを作成して特定の行を参照する
test = pd.DataFrame({"Acolumn": [1, 2, 3], "Bcolumn": [4, 5, 6]})

In [ ]:
#Acolumn列の値が2未満の行だけを参照する
test[test['Acolumn']<2]

In [ ]:
#Acolumn列（シリーズ型）に対して比較演算を行う
test["Acolumn"]<2

In [ ]:
#2つの条件で外れ値の行を特定する
df[(df['SNS2'] > 1000) & (df['sales'] < 8500)]

In [ ]:
#特定した行からインデックスのみを取り出す
no = df2[(df['SNS2']>1000) & (df['sales'] < 8500)].index
no

In [ ]:
#dropメソッドでインデックスが0の行を削除する
test.drop(0,axis=0)

In [ ]:
#列を削除する
test.drop('Bcolumn' , axis=1)

In [ ]:
#dropメソッドを使って外れ値を削除
df3 = df2.drop(no,axis=0)
df3.shape

In [ ]:
#特徴量の列の候補
col = ['SNS1','SNS2','actor','original']
x = df3[col] #特徴量の取り出し

t = df3['sales']

In [ ]:
#インデックスが2、列がSNS1のマスの値のみ参照
df3.loc[2,'SNS1']

In [ ]:
#特定のデータのみを参照する
index = [2,4,6]         #インデックス
col = ['SNS1','actor']  #列名

df3.loc[index,col]

In [ ]:
#スライス構文で連続した要素を参照する
sample = [10,20,30,40]  #リストの作成
sample[1:3] #添え字が1以上3未満の要素を取得

In [ ]:
#0行目以上2行目以下、actor列より左の列（actor列含め）
df3.loc[0:3,:'actor']

In [ ]:
#スライス構文で特徴量と正解データを取り出す
x = df3.loc[:,'SNS1':'original']    #特徴量の取り出し
t = df3['sales']    #正解ラベルの取り出し

In [ ]:
#訓練データとテストデータに分類する
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,t,test_size=0.2,random_state=0)

In [ ]:
#重回帰モデルのLinearRegression関数をインポートする
from sklearn.linear_model import LinearRegression

In [ ]:
#LinearRegression関数を使ってモデルを作成する
model = LinearRegression()

In [ ]:
#fitメソッドでモデルに学習させる
model.fit(x_train,y_train)

In [ ]:
#興行収入を予測する
new = [[150,700,300,0]]
model.predict(new)

In [ ]:
#scoreメソッドでモデルのscoreを計算
model.score(x_test,y_test)

In [ ]:
#MAEを求める
#関数のインポート
from sklearn.metrics import median_absolute_error

pred = model.predict(x_test)

#平均絶対誤差の計算
median_absolute_error(y_pred=pred,y_true=y_test)

In [ ]:
#scoreメソッド
model.score(x_test,y_test)

In [ ]:
#モデルの保存
import pickle

with open("./datafiles/cinema.csv","wb") as f:
    pickle.dump(model,f)

In [ ]:
#係数と切片を確認
print(model.coef_)
print(model.intercept_)

In [ ]:
#列と係数を表示する
tmp = pd.DataFrame(model.coef_)
tmp.index = x_train.columns
tmp